In [2]:
# read in csv
import pandas as pd
df = pd.read_csv('./formated_data.csv')
df.head()

,Unnamed: 0,event_date,landslide_category,landslide_trigger,landslide_size,landslide_setting,fatality_count,injury_count,country_name,country_code,admin_division_name,admin_division_population,longitude,latitude
0,122,2015-05-17 00:00:00,landslide,rain,medium,urban,0.0,0.0,Mexico,MX,Baja California,1173.0,-117.0898,32.5196
1,435,2016-10-01 00:00:00,landslide,downpour,medium,bluff,0.0,0.0,United Kingdom,GB,England,2565.0,0.8546,51.4216
2,437,2015-04-02 00:00:00,landslide,continuous_rain,medium,natural_slope,0.0,0.0,India,IN,Kashmir,14965.0,74.4725,34.1587
3,439,2015-07-19 00:00:00,landslide,downpour,small,above_road,0.0,0.0,United States,US,California,20292.0,-116.9029,33.0742
4,440,2015-10-21 00:00:00,landslide,rain,medium,above_road,0.0,0.0,Mexico,MX,Chiapas,19092.0,-92.2523,15.3482


In [2]:
import requests
import json


def convert_date_to_timestamp(date_string):
    date = pd.to_datetime(date_string)
    return int(date.timestamp())


missing_data = []
def get_temp_on_date(lon, lat, date):
    api_key = 'b8b191c005027d11e69b758a6b29b1e0'
    date = convert_date_to_timestamp(date)
    url = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={date}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = response.json()
    locationData = data['data'][0]
    temp = locationData['temp']
    if(temp is None):
        missing_data.append(locationData)
    return temp


# https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=39.099724&lon=-94.578331&dt=1643803200&appid=b8b191c005027d11e69b758a6b29b1e0


In [22]:
# for each row in df, get the temp on the date of the landslide, and add it to a new column

# try to extract Longitude, Latitude, and Date from the Location column
# if it fails, add it to a list of missing data
# if it succeeds, add the temp to a new column


df['temp'] = df.apply(lambda row: get_temp_on_date(row['longitude'], row['latitude'], row['event_date']), axis=1)
df.head()

# save the new df to a csv
df.to_csv('data_with_temp.csv')


In [17]:
import requests
missing_data = []

def get_temp_on_date_meteo(lon, lat, date):
    dateObject = pd.to_datetime(date)
    hour = dateObject.hour
    print(hour)
    date = dateObject.strftime('%Y-%m-%d')
    # url = f'https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&daily=temperature_2m_max,temperature_2m_min&timezone=Europe%2FLondon'
    # https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2022-10-25&end_date=2022-10-25&hourly=temperature_2m,relativehumidity_2m,windspeed_10m
    url = f'https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&hourly=temperature_2m,relativehumidity_2m,windspeed_10m&timezone=Europe%2FLondon'

    # take hour from date
    # 2015-03-16 14:30:00
    # print(url)
    response = requests.get(url)
    data = response.json()
    # check if the data is missing
    if(data is None):
        return None
    # check if daily exists
    if('hourly' not in data):
        return None
    hourly = data['hourly']
    if(hourly is None):
        return None

    print(hourly)

    if('temperature_2m' not in hourly):
        return None
    temp = hourly['temperature_2m']
    if(temp is None):
        print('temp is none')
        return None

    if('relativehumidity_2m' not in hourly):
        print('relativehumidity_2m not in hourly')
        return None
    humidity = hourly['relativehumidity_2m']

    if('windspeed_10m' not in hourly):
        print('windspeed_10m not in hourly')
        return None
    wind = hourly['windspeed_10m']

    return (temp[hour], humidity[hour], wind[hour])


# print(get_temp_on_date_meteo(-94.578331, 39.099724, '2015-03-16 14:30:00'))

# def merge_into_csv(df):
#     import pandas as pd
#     df = pd.read_csv('./data_with_temp.csv')
#     df = df.append(df)
#     df.to_csv('data_with_temp.csv')


# def get_temp_on_date_meteo_concurrent_thread(df):
    # for index, row in df.iterrows():
    #     temp_min, temp_max = get_temp_on_date_meteo(row['longitude'], row['latitude'], row['event_date'])
    #     df.at[index, 'temp_min'] = temp_min
    #     df.at[index, 'temp_max'] = temp_max
    #     print(temp_min, temp_max)
    # merge_into_csv(df)

        # print(temp_min, temp_max)

# print(get_temp_on_date_meteo(107.4500, 32.5625, '2008-08-01'))

# get_temp_on_date_meteo_concurrent_thread(df.head())

14
{'time': ['2015-03-16T00:00', '2015-03-16T01:00', '2015-03-16T02:00', '2015-03-16T03:00', '2015-03-16T04:00', '2015-03-16T05:00', '2015-03-16T06:00', '2015-03-16T07:00', '2015-03-16T08:00', '2015-03-16T09:00', '2015-03-16T10:00', '2015-03-16T11:00', '2015-03-16T12:00', '2015-03-16T13:00', '2015-03-16T14:00', '2015-03-16T15:00', '2015-03-16T16:00', '2015-03-16T17:00', '2015-03-16T18:00', '2015-03-16T19:00', '2015-03-16T20:00', '2015-03-16T21:00', '2015-03-16T22:00', '2015-03-16T23:00'], 'temperature_2m': [20.7, 18.4, 16.5, 14.8, 14.3, 13.6, 13.5, 13.4, 13.0, 12.7, 11.1, 11.1, 11.0, 10.8, 13.4, 16.9, 19.8, 22.4, 23.9, 24.9, 25.8, 26.5, 25.4, 24.9], 'relativehumidity_2m': [32, 36, 40, 45, 47, 54, 58, 65, 72, 82, 86, 87, 89, 88, 80, 59, 49, 43, 41, 40, 37, 36, 37, 38], 'windspeed_10m': [13.5, 15.0, 16.3, 18.0, 17.9, 18.3, 19.7, 19.1, 18.1, 17.8, 16.3, 16.1, 15.3, 14.8, 16.1, 15.9, 16.1, 16.5, 18.8, 21.4, 23.0, 24.2, 20.5, 18.2]}
(13.4, 80, 16.1)


In [18]:
# for each row in df, get the temp on the date of the landslide, and add it to a new column
# use get_temp_on_date_meteo

# try to extract Longitude, Latitude, and Date from the Location column
# if it fails, add it to a list of missing data
# if it succeeds, add the temp to a new column

import json
# json_output = []
# json_file = open('data.json', 'w')

for index, row in df.iterrows():
    # print(index)
    temp = get_temp_on_date_meteo(row['longitude'], row['latitude'], row['event_date'])
    if(temp is None):
        missing_data.append(row)
    else:
        df.at[index, 'temp'] = temp[0]
        df.at[index, 'humidity'] = temp[1]
        df.at[index, 'wind'] = temp[2]
        # open, read in, append, write out
        # json_read_file = open('data.json', 'r')
        # print(json_read_file)
        # load json from data.json
        json_read_file = open('data_formatted.json')
        json_data = json.load(json_read_file)
        # json_data = json.load(json_read_file)
        # print(json_data)
        json_read_file.close()

        json_write_file = open('data_formatted.json', 'w')

        dfRow = df.iloc[index].to_dict()
        json_data.append(dfRow)
        # dump json
        json.dump(json_data, json_write_file)
        json_write_file.close()



# print(missing_data)
# df['temp_min'] = df.apply(lambda row: get_temp_on_date_meteo(row['longitude'], row['latitude'], row['event_date'])[0], axis=1)
# df['temp_max'] = df.apply(lambda row: get_temp_on_date_meteo(row['longitude'], row['latitude'], row['event_date'])[1], axis=1)
# df.head()

# save the new df to a csv
df.to_csv('data_with_temp_meteo.csv')

0
{'time': ['2015-05-17T00:00', '2015-05-17T01:00', '2015-05-17T02:00', '2015-05-17T03:00', '2015-05-17T04:00', '2015-05-17T05:00', '2015-05-17T06:00', '2015-05-17T07:00', '2015-05-17T08:00', '2015-05-17T09:00', '2015-05-17T10:00', '2015-05-17T11:00', '2015-05-17T12:00', '2015-05-17T13:00', '2015-05-17T14:00', '2015-05-17T15:00', '2015-05-17T16:00', '2015-05-17T17:00', '2015-05-17T18:00', '2015-05-17T19:00', '2015-05-17T20:00', '2015-05-17T21:00', '2015-05-17T22:00', '2015-05-17T23:00'], 'temperature_2m': [16.4, 16.3, 16.0, 15.7, 15.5, 15.4, 15.3, 15.4, 15.1, 15.1, 14.2, 14.1, 14.2, 14.3, 14.7, 15.3, 16.0, 16.8, 17.7, 17.8, 17.8, 17.7, 17.4, 17.0], 'relativehumidity_2m': [74, 74, 76, 78, 79, 80, 80, 79, 81, 82, 84, 84, 84, 84, 82, 77, 74, 71, 71, 73, 75, 75, 76, 78], 'windspeed_10m': [13.6, 11.0, 10.0, 6.0, 1.9, 0.8, 2.6, 4.1, 4.4, 3.1, 5.1, 5.5, 3.9, 3.8, 4.6, 6.0, 6.5, 7.2, 10.5, 13.8, 15.1, 15.8, 14.8, 13.8]}
0
{'time': ['2016-10-01T00:00', '2016-10-01T01:00', '2016-10-01T02:00', '2

In [6]:
# threading
import threading
import time
import numpy as np

def apply_all(df):
    chunks = np.array_split(df, 30)
    import concurrent.futures
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        # thread for each chunk
        for chunk in chunks:
            executor.submit(get_temp_on_date_meteo_concurrent_thread, chunk)
            
        
        
start_time = time.time()
apply_all(df)
print("--- %s seconds ---" % (time.time() - start_time))


# save the new df to a csv
# df.to_csv('data_with_temp_meteo_threaded.csv')

https://archive-api.open-meteo.com/v1/era5?latitude=16.574445&longitude=120.732714&start_date=2016-08-30&end_date=2016-08-30&daily=temperature_2m_max,temperature_2m_min&timezone=Europe%2FLondonhttps://archive-api.open-meteo.com/v1/era5?latitude=32.5625&longitude=107.45&start_date=2008-08-01&end_date=2008-08-01&daily=temperature_2m_max,temperature_2m_min&timezone=Europe%2FLondon
https://archive-api.open-meteo.com/v1/era5?latitude=49.0099&longitude=-58.1315&start_date=2016-10-11&end_date=2016-10-11&daily=temperature_2m_max,temperature_2m_min&timezone=Europe%2FLondon

https://archive-api.open-meteo.com/v1/era5?latitude=45.2771&longitude=-123.8289&start_date=2012-03-02&end_date=2012-03-02&daily=temperature_2m_max,temperature_2m_min&timezone=Europe%2FLondon
https://archive-api.open-meteo.com/v1/era5?latitude=47.3848&longitude=-122.4801&start_date=2010-10-10&end_date=2010-10-10&daily=temperature_2m_max,temperature_2m_min&timezone=Europe%2FLondon
https://archive-api.open-meteo.com/v1/era5?lat

In [20]:
def json_to_csv():
    import json
    import pandas as pd
    json_read_file = open('data_formatted.json')
    json_data = json.load(json_read_file)
    json_read_file.close()
    df = pd.DataFrame(json_data)
    df.to_csv('data_formatted_in_csv.csv')

json_to_csv()
